In [ ]:
import arcpy, math
import os
import os.path
from arcpy import env
import random
from arcpy.sa import *
import pandas as pd
import numpy as np
# Check out any necessary licenses
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
import matplotlib.pyplot as plt

In [ ]:
folder_rastry_legacies = r"c:\Users\marci\Dysk Google\jupyter_pyton\CZEMP\dane\wzorcowy_raw"
folder_modeli = r'c:\doktorat\czemp6\powiaty'
folder_out = folder_modeli

In [ ]:
arcpy.env.workspace = folder_rastry_legacies
lista_rastrow = arcpy.ListRasters()
lista_rastrow2 = []

for i in lista_rastrow:
    desc = arcpy.Describe(i)
    sciezka = os.path.join(desc.path, desc.name)
    lista_rastrow2.append(sciezka)

arcpy.env.workspace = folder_modeli
lista_modeli = arcpy.ListFeatureClasses()
foldery_modeli=[]

for model in lista_modeli:
    print(model)
    nazwa = model[:-4]
    arcpy.env.extent = model
    sciezka_model = os.path.join(folder_out, nazwa)
    try:
        os.mkdir(sciezka_model)
        foldery_modeli.append(sciezka_model)
    except:
        print("nie robi foldera, pewnie juz byl")
    for raster in lista_rastrow2:
        raster_name = os.path.basename(raster)
        outExtractByMask1 = ExtractByMask(raster, model)
        outExtractByMask1.save(os.path.join(sciezka_model, raster_name))

In [2]:
for model_folder in foldery_modeli:
        data_frame = pd.DataFrame()
        arcpy.env.workspace = model_folder
        os.chdir(model_folder)
        lista_rastrow = arcpy.ListRasters()
        arcpy.env.overwriteOutput = True
        lista_kolumn =[]
        for j in lista_rastrow:
            try:
                array1 = arcpy.RasterToNumPyArray(j)
                plt.imshow(array1)
                plt.show()
                plt.clf()
                array_plaska = array1.ravel()
                series = pd.Series(array_plaska)
                kolumna = str(j[:-4])
                data_frame[kolumna] = series.values
                lista_kolumn.append(kolumna)
            except:
                print("dasd")

            df_skrocony2 = data_frame.sample(frac=0.25, replace=True)
            df_skrocony = df_skrocony2.replace({255: np.NaN, -32768.000000: np.NaN, -9999: np.NaN})
            df3 = df_skrocony.dropna()
            csv = os.path.join(folder_nadrzedny, str(model_folder) + ".csv")
            print(csv)
            df3.to_csv(csv)